<a href="https://colab.research.google.com/github/totti0223/deep_learning_for_biologists_with_keras/blob/master/notebooks/1_deepyeast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yeast GFP protein localization classification with CNN

![yeast_image](https://github.com/totti0223/deep_learning_for_biologists_with_keras/raw/master/assets/yeast.jpg)
Above image adopted from http://kodu.ut.ee/~leopoldp/2016_DeepYeast/

##Reference
*Accurate Classification of Protein Subcellular Localization from High-Throughput Microscopy Images Using Deep Learning
Tanel Pärnamaa and Leopold Parts
G3: GENES, GENOMES, GENETICS May 1, 2017 vol. 7 no. 5 1385-1392; https://doi.org/10.1534/g3.116.033654*

http://kodu.ut.ee/~leopoldp/2016_DeepYeast/

# Import Libraries

In [1]:
import csv
import numpy as np
import os
import math

import itertools
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix
from skimage.exposure import equalize_adapthist

from skimage.io import imread
from skimage.transform import resize

import keras
import keras.backend as K
from keras.utils import np_utils
from keras.utils.data_utils import get_file

from keras import layers,models
from keras.models import Sequential


Using TensorFlow backend.


In [2]:
#update imagegenerator function
!pip uninstall keras-preprocessing -y
!pip install git+https://github.com/keras-team/keras-preprocessing.git

Uninstalling Keras-Preprocessing-1.0.5:
  Successfully uninstalled Keras-Preprocessing-1.0.5
  Cloning https://github.com/keras-team/keras-preprocessing.git to /tmp/pip-req-build-h2wz77uy
  Running setup.py bdist_wheel for Keras-Preprocessing ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-kkep4d3b/wheels/03/a0/39/171f6040d36f36c71168dc69afa81334351b20955dc36ce932
Successfully built Keras-Preprocessing


In [0]:
from keras_preprocessing.image import ImageDataGenerator as _ImageDataGenerator
#from keras.preprocessing.image import ImageDataGenerator

# Prepare Data

## Download yeast GFP datasets

In [5]:
label_names = ['Cell_periphery','Cytoplasm','endosome','ER','Golgi','Mitochondrion','Nuclear_Periphery','Nucleolus','Nucleus','Peroxisome','Spindle_pole','Vacuole']


def download_data():
    print("Downloading dataset")
    paths = ["main.tar.gz","HOwt_train.txt","HOwt_val.txt","HOwt_test.txt"]
    data_path = get_file(paths[0],origin="http://kodu.ut.ee/~leopoldp/2016_DeepYeast/data/main.tar.gz",extract=True,cache_subdir='deepyeast')
    train_path = get_file(paths[1],origin="http://kodu.ut.ee/~leopoldp/2016_DeepYeast/code/reports/HOwt_train.txt",cache_subdir='deepyeast')
    val_path = get_file(paths[2],origin="http://kodu.ut.ee/~leopoldp/2016_DeepYeast/code/reports/HOwt_val.txt",cache_subdir='deepyeast')
    test_path = get_file(paths[3],origin="http://kodu.ut.ee/~leopoldp/2016_DeepYeast/code/reports/HOwt_test.txt",cache_subdir='deepyeast')
    return data_path,train_path,val_path,test_path

def download_transfer_data():
    print("Downloading dataset for transfer learning")
    paths = ["transfer.tar.gz","HOwt_transfer_train.txt","HOwt_transfer_val.txt","HOwt_transfer_test.txt"]

    data_path = get_file(paths[0], origin='http://kodu.ut.ee/~leopoldp/2016_DeepYeast/data/transfer.tar.gz',extract=True,cache_subdir='deepyeast_transfer')   
    train_path = get_file(paths[1], origin='http://kodu.ut.ee/~leopoldp/2016_DeepYeast/code/image_prep/data/HOwt_transfer_train.txt', cache_subdir='deepyeast_transfer')
    val_path = get_file(paths[2], origin='http://kodu.ut.ee/~leopoldp/2016_DeepYeast/code/image_prep/data/HOwt_transfer_val.txt', cache_subdir='deepyeast_transfer')
    test_path = get_file(paths[3], origin='http://kodu.ut.ee/~leopoldp/2016_DeepYeast/code/image_prep/data/HOwt_transfer_test.txt', cache_subdir='deepyeast_transfer')

data_path, train_path, val_path, test_path = download_data()

print(data_path)
print(train_path)
print(val_path)
print(test_path)

/root/.keras/deepyeast/main.tar.gz
/root/.keras/deepyeast/HOwt_train.txt
/root/.keras/deepyeast/HOwt_val.txt
/root/.keras/deepyeast/HOwt_test.txt


## Read the CSV as pandas dataframe

In [6]:
parent_directory = "/root/.keras/deepyeast/"

train_df = pd.read_csv(train_path,delimiter=" ",names=("filename","class"),dtype={"filename":"str","class":"int"})
train_df["filename"] = parent_directory + train_df["filename"]

valid_df = pd.read_csv(val_path,delimiter=" ",names=("filename","class"),dtype={"filename":"str","class":"int"})
valid_df["filename"] = parent_directory + valid_df["filename"]

test_df = pd.read_csv(test_path,delimiter=" ",names=("filename","class"),dtype={"filename":"str","class":"int"})
test_df["filename"] = parent_directory + test_df["filename"]


train_df[:5]

,filename,class
0,/root/.keras/deepyeast/plate10/015005000-6-151...,0
1,/root/.keras/deepyeast/plate01/007013000-2-626...,6
2,/root/.keras/deepyeast/plate01/003018000-2-262...,1
3,/root/.keras/deepyeast/plate10/004020000-0-141...,3
4,/root/.keras/deepyeast/plate08/006018000-0-113...,1


## Create ImageDataGenerator Class

In [7]:
train_datagen = _ImageDataGenerator(rescale=1/255.)
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                  directory=None,
                                  x_col="filename",
                                  y_col="class",
                                  has_ext = True,
                                  target_size=(64,64),
                                  class_mode="categorical")


valid_datagen = _ImageDataGenerator(rescale=1/255.)
valid_generator = valid_datagen.flow_from_dataframe(dataframe=valid_df,
                                  directory=None,
                                  x_col="filename",
                                  y_col="class",
                                  has_ext = True,
                                  target_size=(64,64),
                                  class_mode="categorical")

test_datagen = _ImageDataGenerator(rescale=1/255.)
test_generator = test_datagen.flow_from_dataframe(dataframe=test_df,
                                  directory=None,
                                  x_col="filename",
                                  y_col="class",
                                  has_ext = True,
                                  target_size=(64,64),
                                  class_mode="categorical")

Found 65000 images belonging to 12 classes.
Found 12500 images belonging to 12 classes.
Found 12500 images belonging to 12 classes.


# Model preparation

## Create the CNN model

In [0]:
#basically follows the CNN network of the cited literature.

model = Sequential([
    #feature extraction layer
    
    #block1
    layers.Conv2D(64,(3,3),padding="same",name="block1_conv1",input_shape=(64,64,3)),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Conv2D(64,(3,3),padding="same",name="block1_conv2"),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.MaxPooling2D((2,2),strides=(2,2),name="block1_pool"),
    #block2
    layers.Conv2D(128,(3,3),padding="same",name="block2_conv1"),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Conv2D(128,(3,3),padding="same",name="block2_conv2"),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.MaxPooling2D((2,2),strides=(2,2),name="block2_pool"),
    #block3
    layers.Conv2D(256,(3,3),padding="same",name="block3_conv1"),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Conv2D(256,(3,3),padding="same",name="block3_conv2"),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Conv2D(256,(3,3),padding="same",name="block3_conv3"),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Conv2D(256,(3,3),padding="same",name="block3_conv4"),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.MaxPooling2D((2,2),strides=(2,2),name="block3_pool"),

    layers.Flatten(),
    
    #inference layer
    layers.Dense(512,name="fc1"),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Dropout(0.5),
    
    layers.Dense(512,name="fc2"),
    layers.BatchNormalization(),
    layers.Activation("relu"),    
    layers.Dropout(0.5),
    
    layers.Dense(12,name="prepredictions"),
    layers.Activation("softmax",name="predictions")
    
])

model.compile(optimizer = "adam", loss="categorical_crossentropy", metrics=["accuracy"])


##check the model architecture by model.summary

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 64)        0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
activation_2 (Activation)    (None, 64, 64, 64)        0         
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
__________

# Train the CNN

In [0]:
#limiting the epochs to two. if you have time, change it to larger number and see how well it improves
model.fit_generator(generator=train_generator,
                    steps_per_epoch= train_generator.n//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.n//valid_generator.batch_size,
                    epochs=2)

Epoch 1/2
2031/2031 [==============================] - 262s 129ms/step - loss: 1.0030 - acc: 0.6667 - val_loss: 1.3184 - val_acc: 0.6189
Epoch 2/2
2031/2031 [==============================] - 257s 127ms/step - loss: 0.5191 - acc: 0.8307 - val_loss: 1.1925 - val_acc: 0.6370


## Load the pretrained model 

In [10]:
#Lets download the pretrained model for people who cannot wait for the model to be sufficiently trained
!wget https://raw.githubusercontent.com/totti0223/deep_learning_for_biologists_with_keras/master/notebooks/1_yeast_best_model.hdf5 -O 1_yeast_best_model.hdf5
    
#Load weights into the model
model.load_weights("1_yeast_best_model.hdf5")

--2018-12-24 07:16:21--  https://raw.githubusercontent.com/totti0223/deep_learning_for_biologists_with_keras/master/notebooks/1_yeast_best_model.hdf5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88055192 (84M) [application/octet-stream]
Saving to: ‘1_yeast_best_model.hdf5’

1_yeast_best_model. 100%[===================>]  83.98M   148MB/s    in 0.6s    

2018-12-24 07:16:25 (148 MB/s) - ‘1_yeast_best_model.hdf5’ saved [88055192/88055192]



## Evaluate

In [15]:
#lets see what kind of metrics can be obtained
print(model.metrics_names) 

#evaluate train dataset, validation dataset, and test dataset

print("train dataset: ", end = "")
print(model.evaluate_generator(train_generator, steps = train_generator.n//train_generator.batch_size))

print("validation dataset: ", end = "")
print(model.evaluate_generator(valid_generator, steps = valid_generator.n//valid_generator.batch_size))

print("test dataset: ", end = "")
print(model.evaluate_generator(test_generator, steps = test_generator.n//test_generator.batch_size))

['loss', 'acc']
train dataset: [0.10417130680581842, 0.9644748183721217]
validation dataset: [0.4137479445156761, 0.8745192307692308]
test dataset: [0.4667509896441912, 0.8633814102564102]


from the above evaluation, we can confirm that the classifier has 86.3% accuracy with 0.467 loss in the test dataset.
more accuracy can be observed by modifying the CNN, training optimizer, and/or performing dataaugmentation in training dataset. try yourself!

In [0]:
label_name = deepyeast.load_label_names() #label name that corresponds to the label number in dataset
true_y = np.argmax(y_test,axis=1) #the true label of dataset
pred_y = np.argmax(model.predict(X_test),axis=1) #predicted label of dataset

evaluation summary function by sklearn.

In [0]:
print(classification_report(true_y,pred_y,target_names=label_name))

                   precision    recall  f1-score   support

   Cell_periphery       0.97      0.87      0.92      1569
        Cytoplasm       0.84      0.92      0.88      1276
         endosome       0.62      0.83      0.71       689
               ER       0.89      0.92      0.91      1755
            Golgi       0.92      0.82      0.87       382
    Mitochondrion       0.89      0.90      0.90      1243
Nuclear_Periphery       0.94      0.89      0.92      1164
        Nucleolus       0.85      0.92      0.89      1263
          Nucleus       0.95      0.82      0.88      1627
       Peroxisome       0.53      0.81      0.64       164
     Spindle_pole       0.81      0.57      0.67       781
          Vacuole       0.74      0.88      0.80       587

      avg / total       0.87      0.86      0.86     12500



let's get the confusion matrix, also using the sklearn function

In [0]:
import kwb.utils

cnf = confusion_matrix(true_y, pred_y)
np.set_printoptions(precision=2)

plt.figure(figsize=(8,8))
kwb.utils.plot_confusion_matrix(cnf, classes=label_name,
                      title='Confusion matrix',normalize=True)

NameError: name 'confusion_matrix' is not defined

In [0]:
X_train, y_train, X_valid, y_valid,X_test,y_test = load_data(transfer=True)
print(X_train.shape,y_train.shape,X_valid.shape,y_valid.shape,)

/home/dl-box/anaconda3/envs/keras/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


(4000, 64, 64, 3) (4000, 4) (2000, 64, 64, 3) (2000, 4)


In [0]:
#transfer learning with random forest.
from keras.models import Model
from keras import layers
from sklearn.ensemble import RandomForestClassifier

for layer in model.layers:
    layer.trainable = False    
intlayer = model.get_layer(index=-2).output #get value before the last layer
intmodel = Model(inputs=model.input,outputs=intlayer)

int_X_train = intmodel.predict(X_train)
int_X_test = intmodel.predict(X_test)
print(int_X_train.shape,int_X_test.shape)

clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
clf.fit(int_X_train,np.argmax(y_train,axis=1))
print("accuracy on train data:", clf.score(int_X_train,np.argmax(y_train,axis=1)))
print("accuracy on test data:", clf.score(int_X_test,np.argmax(y_test,axis=1)))

accuracy on train data: 0.7155
accuracy on test data: 0.61475


In [0]:
#another transfer learning that connects new dense layer to the learnt network
import keras.backend as K

K.clear_session()

model =DeepYeast()
model.load_weights("DeepYeast_Bestmodel.hdf5")
for layer in model.layers:
    layer.trainable=False
intlayer = model.get_layer(index=-2).output #get value before the last layer

x = layers.Dense(4, activation='softmax', name='predictions')(intlayer)

transfermodel = Model(inputs=model.input,outputs=x)
transfermodel.summary() #note that trainable parameters are limited to dense layers only
transfermodel.compile("sgd",loss="categorical_crossentropy",metrics=["acc"])

bestmodel = keras.callbacks.ModelCheckpoint("DeepYeast_Bestmodel_transfer.hdf5",save_best_only=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 64)        0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
activation_2 (Activation)    (None, 64, 64, 64)        0         
__________

In [0]:
transfermodel.fit(X_train, y_train,
                    epochs=50,batch_size = 100,callbacks=[bestmodel],
                   validation_data=(X_valid,y_valid))

Train on 4000 samples, validate on 2000 samples
Epoch 1/50
4000/4000 [==============================] - 2s 539us/step - loss: 1.5590 - acc: 0.4062 - val_loss: 1.2991 - val_acc: 0.4590
Epoch 2/50
4000/4000 [==============================] - 2s 429us/step - loss: 1.1935 - acc: 0.5527 - val_loss: 1.2290 - val_acc: 0.4830
Epoch 3/50
4000/4000 [==============================] - 2s 444us/step - loss: 1.1040 - acc: 0.5940 - val_loss: 1.1880 - val_acc: 0.5135
Epoch 4/50
4000/4000 [==============================] - 2s 431us/step - loss: 1.0418 - acc: 0.6100 - val_loss: 1.1516 - val_acc: 0.5360
Epoch 5/50
4000/4000 [==============================] - 2s 438us/step - loss: 1.0395 - acc: 0.6255 - val_loss: 1.1372 - val_acc: 0.5465
Epoch 6/50
4000/4000 [==============================] - 2s 440us/step - loss: 0.9982 - acc: 0.6468 - val_loss: 1.1202 - val_acc: 0.5505
Epoch 7/50
4000/4000 [==============================] - 2s 444us/step - loss: 0.9758 - acc: 0.6480 - val_loss: 1.1049 - val_acc: 0.5595


In [0]:
print(transfermodel.evaluate(X_train,y_train))
print(transfermodel.evaluate(X_valid,y_valid))
print(transfermodel.evaluate(X_test,y_test))

4000/4000 [==============================] - 1s 362us/step
[0.7705937781333924, 0.76175]
2000/2000 [==============================] - 1s 365us/step
[1.1149951038360595, 0.571]
4000/4000 [==============================] - 1s 357us/step
[0.9392598533630371, 0.669]


In [0]:
#with recent model
K.clear_session()
